In [1]:
import pandas as pd
import time
import sys
def player_data(start_year,end_year,ps=False):
    if ps == False:
        season_type='leagues'
    else:
        season_type='playoffs'
    frames=[]
    for year in range(start_year,end_year+1):
        url = 'https://www.basketball-reference.com/'+season_type+'/NBA_'+str(year)+'_per_poss.html'
        df=pd.read_html(url)[0]
        df=df[df.Player!='Player']
        time.sleep(1)
        url2='https://www.basketball-reference.com/'+season_type+'/NBA_'+str(year)+'_advanced.html'
        df2=pd.read_html(url2)[0]
        df2=df2[df2.Player!='Player']

        

        year_df=df.merge(df2)
        unnamed=[col for col in year_df if 'unnamed' in col.lower()]
        year_df.drop(columns=unnamed,inplace=True)
        year_df['year']=year
        frames.append(year_df)

        time.sleep(1.5)
    return pd.concat(frames)
start_year=2000
end_year=2024
player_stats=player_data(start_year,end_year)

player_stats.to_csv('player_stats.csv',index=False)
player_stats_ps=player_data(start_year,end_year,ps=True)
player_stats.to_csv('player_stats_ps.csv',index=False)


496
496
496
537
537
537
500
500
500
483
483
483
585
585
585
585
585
585
563
563
563
516
516
516
595
595
595
582
582
582
578
578
578
625
625
625
551
551
551
573
573
573
611
611
611
651
651
651
578
578
578
595
595
595
664
664
664
708
708
708
651
651
651
705
705
705
812
812
812
679
679
679
735
735
735
181
181
181
190
190
190
186
186
186
187
187
187
189
189
189
188
188
188
195
195
195
191
191
191
202
202
202
199
199
199
199
199
199
194
194
194
204
204
204
209
209
209
204
204
204
208
208
208
215
215
215
215
215
215
210
210
210
212
212
212
217
217
217
239
239
239
217
217
217
217
217
217
214
214
214


In [2]:
def league_average(league='leagues'):
    time.sleep(1)
    league_url='https://www.basketball-reference.com/'+league+'/NBA_stats_per_poss.html'
    frames=pd.read_html(league_url)
    league = frames[0]
    league.columns=league.columns.droplevel()
    league=league[league.Season!='Season']

    league2 = frames[1]
    league2.columns=league2.columns.droplevel()
    league2=league2[league2.Season!='Season']
    
    
    return league,league2
league,league2 = league_average()
league.to_csv('league_average.csv',index=False)

league2.to_csv('league_average_ps.csv',index=False)



In [3]:
league2

,Rk,Season,Lg,Age,Ht,Wt,G,FG,FGA,3P,...,FG%,3P%,FT%,Pace,eFG%,TOV%,ORB%,FT/FGA,ORtg,TS%
0,1,2023-24,NBA,26.4,6-7,216,82,41.9,90.7,12.6,...,.462,.351,.779,92.7,.532,11.6,24.1,.188,113.5,.566
1,2,2022-23,NBA,26.1,6-6,216,84,41.6,90.3,12.5,...,.461,.351,.794,96.1,.530,11.7,24.0,.197,113.5,.566
2,3,2021-22,NBA,26.1,6-6,215,87,39.9,87.6,12.8,...,.456,.355,.785,95.5,.529,12.8,22.3,.212,111.2,.567
3,4,2020-21,NBA,26.1,6-6,217,85,41.6,90.0,12.9,...,.462,.364,.786,96.1,.534,11.2,22.8,.201,114.3,.571
4,5,2019-20,NBA,26.1,6-6,218,83,39.4,86.3,13.3,...,.457,.360,.788,97.6,.534,12.8,20.4,.221,111.3,.574
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,74,1950-51,NBA,NaN,NaN,NaN,26,NaN,NaN,NaN,...,.376,NaN,.723,NaN,.376,NaN,NaN,.308,NaN,.446
80,75,1949-50,NBA,NaN,NaN,NaN,30,NaN,NaN,NaN,...,.344,NaN,.732,NaN,.344,NaN,NaN,.343,NaN,.427
81,76,1948-49,BAA,NaN,NaN,NaN,20,NaN,NaN,NaN,...,.329,NaN,.717,NaN,.329,NaN,NaN,.306,NaN,.405
82,77,1947-48,BAA,NaN,NaN,NaN,21,NaN,NaN,NaN,...,.272,NaN,.701,NaN,.272,NaN,NaN,.244,NaN,.342
